In [1]:
import sys
sys.path.append('.')

import torch
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

In [2]:
train_df: pd.DataFrame = pd.read_pickle('data/train.pkl')

In [3]:
missions = train_df[['ID', 'mission']] \
    .drop_duplicates() \
    .sort_index()

missions['type'] = missions['mission'].str.split('_').str[0]
missions['target'] = missions['mission'].str.split('_').str[1]
missions.drop(columns=['mission'], inplace=True)
missions = missions.to_dict(orient='records')

In [4]:
from src.gru4rec import GRU4Rec, SequentialRecommendations, train

model = GRU4Rec(n_items=len(missions), hidden_size=16, n_layers=2)
dataset = SequentialRecommendations(train_df)
model = train(model, dataset, n_epochs=10, lr=1e-4)

torch.save(model.state_dict(), 'models/gru4rec.pth')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:00<?, ?it/s]

...

  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:00<?, ?it/s]